In [3]:
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter, FuncFormatter
import torch, pickle, sys
sys.path.append('/home/dgotzens/scripts/')
import pdfdefaults as pdf
# setup
nfft = 2**12

pi = 3.141592653589 
maxdist = 50
lightspeed = 299_792_458 
f_start, f_end = 76_009_996_288.0, 80_289_505_280.0
t_chirp = 0.000_064_890_002_249_740_060
omega0 = 2*pi*f_start
omegadot = 2*pi*(f_end-f_start) / t_chirp
hertz_per_meter = (f_end-f_start) / t_chirp / lightspeed
bins_per_meter = nfft / maxdist

folder = '/home/dgotzens/scripts/2.2_antenna-characteristics/measured/'
params = pickle.load(open(folder+'feparams.pkl', 'rb'))
tx,ty,rx,ry = params['txPosX'], params['txPosY'], params['rxPosX'], params['rxPosY']

In [4]:
dists = (2,8,18,32)
N = len(dists)
K = 192
maxgain = torch.empty(K,N)
exact_dists = torch.empty(K,N)
folder = '/home/dgotzens/recording/'

for n,dist in enumerate(dists):
    data = torch.load(f'{folder}a{dist:02d}_data.pt')
    M,K,L = data.shape
    print(f'loaded data for {dist}m. processing...')

    bp_start = int((dist-0.5)*bins_per_meter)
    bp_len = int(1*bins_per_meter)

    window = torch.hann_window(M)
    window = window / window.sum()

    gain = torch.empty(K,L)
    m_refl = torch.empty(K,L, dtype=int)
    for l in range(L):
        fft = torch.fft.fft(window[:,None]*data[:,:,l], n=nfft, dim=0)
        m_refl[:,l] = fft[bp_start:bp_start+bp_len,:].abs().argmax(0) + bp_start
        gain[:,l] = fft.abs()[m_refl[:,l], range(K)]
    maxgain[:,n] = gain.max(1).values
    idx = gain.max(1).indices
    exact_dists[:,n] = m_refl[range(K),idx]/bins_per_meter

loaded data for 2m. processing...
loaded data for 8m. processing...
loaded data for 18m. processing...
loaded data for 32m. processing...


In [5]:
pdf.setup()
# %matplotlib inline
attenuation = lambda d : lightspeed**2 / (2*d**2*(omega0*lightspeed + omegadot*2*d)) 
gain_1m = (attenuation(exact_dists)*maxgain).sum(1) / ((attenuation(exact_dists))**2).sum(1)

k=gain_1m.argmax()
plt.figure(figsize=(0.5*pdf.a4_textwidth, 0.4*pdf.a4_textwidth), layout='constrained')
plt.plot(torch.arange(1.5,35,0.5), gain_1m[k]*attenuation(torch.arange(1.5,35,0.5)), 'k:', label='data')
plt.scatter(exact_dists[k,:], maxgain[k,:], label='regression')
plt.title('Linear Regression for channel 0')
plt.gca().xaxis.set_major_formatter(EngFormatter('m'))
plt.grid()
plt.xlabel('distance')
plt.ylabel('amplitude')
# plt.show()
plt.savefig('/home/dgotzens/thesis/figures/ch0_amplitude_linreg.pdf')


In [6]:
Ntx,Nrx = 12,16
gainmatrix = gain_1m.unflatten(0,(Ntx,Nrx))
mse = (gain_1m[:,None]*attenuation(exact_dists) - maxgain).square().mean(1)
# %matplotlib inline
pdf.setup()

from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, (left,right) = plt.subplots(1,2,layout='tight')
fig.set_size_inches(pdf.a4_textwidth,0.4*pdf.a4_textwidth)
im1 = left.pcolormesh(gainmatrix)
left.set_yticks(range(0,Ntx+1,3))
left.set_xticks(range(0,Nrx+1,4))
left.set_ylabel('tx-id')
left.set_xlabel('rx-id')
left.set_title('Extrapolated Transmission Coefficients')
left.set_aspect('equal')
left.grid()

divider = make_axes_locatable(left)
cax = divider.append_axes('right', size='5%', pad=0.1)
fig.colorbar(im1 , cax=cax, orientation='vertical')

im2 = right.pcolormesh(mse.unflatten(0,(Ntx,Nrx)))
right.grid()
right.set_yticks(range(0,Ntx+1,3))
right.set_xticks(range(0,Nrx+1,4))
right.set_ylabel('tx-id')
right.set_xlabel('rx-id')
right.set_title('Mean Square Error')
right.set_aspect('equal')

divider = make_axes_locatable(right)
cax = divider.append_axes('right', size='5%', pad=0.1)
fig.colorbar(im2, cax=cax, orientation='vertical')

fig.savefig('/home/dgotzens/thesis/figures/amplitude_linreg.pdf')

In [10]:
Atx = gainmatrix[:,0]**2
Arx = (gainmatrix**2/Atx[:,None]).mean(0)

print([f'{float(a):.3e}' for a in Atx])
print([f'{float(a):.3e}' for a in Arx])

fig, (left,right) = plt.subplots(1,2,layout='constrained')
fig.set_size_inches(0.8*pdf.a4_textwidth, 0.5*pdf.a4_textwidth)
left.stem(range(Ntx),Atx)
left.set_ylabel('gain')
left.set_xlabel('tx-id')
left.set_xticks(range(0,Ntx+1,Ntx//4))
left.set_title('transmit antenna gain')
left.grid()

right.stem(range(Nrx),Arx)
right.set_ylabel('gain')
right.set_xlabel('rx-id')
right.set_xticks(range(0,Nrx+1,Nrx//4))
right.set_title('receive antenna gain')
right.grid()

fig.savefig('/home/dgotzens/thesis/figures/antenna_gains.pdf')

['4.134e+06', '7.847e+06', '7.363e+06', '6.169e+06', '5.445e+06', '6.815e+06', '5.174e+06', '3.962e+06', '3.517e+06', '2.831e+06', '2.291e+06', '2.158e+06']
['1.000e+00', '1.463e+00', '1.676e+00', '1.426e+00', '3.797e+00', '3.635e+00', '3.999e+00', '3.703e+00', '3.391e+00', '2.867e+00', '2.751e+00', '2.631e+00', '1.393e+00', '9.871e-01', '1.129e+00', '8.201e-01']


In [17]:
txgain = ['4.134e+06', '7.847e+06', '7.363e+06', '6.169e+06', '5.445e+06', '6.815e+06', '5.174e+06', '3.962e+06', '3.517e+06', '2.831e+06', '2.291e+06', '2.158e+06']
rxgain = ['1.000', '1.463', '1.676', '1.426', '3.797', '3.635', '3.999', '3.703', '3.391', '2.867', '2.751', '2.631', '1.393', '0.987', '1.129', '0.820']
txphi = ['53.596', '125.805', '48.354', '-126.556', '-13.976', '-111.434', '-81.855', '82.760', '119.322', '1.485', '7.703', '-53.846']
rxphi = ['0.000', '-7.095', '-26.095', '0.712', '-1.052', '26.672', '42.318', '-2.503', '49.492', '-2.698', '69.663', '-40.625', '8.481', '-11.289', '-1.515', '-5.109']



for i, (gain,angl) in enumerate(zip(txgain,txphi)):
    print(str(i) + r' & \num{' + gain + r'} & ' + angl  +r'°  \\')

print()

for i, (gain,angl) in enumerate(zip(rxgain,rxphi)):
    print(str(i) + r' & \num{' + gain + r'} & ' + angl  +r'°  \\')

0 & \num{4.134e+06} & 53.596°  \\
1 & \num{7.847e+06} & 125.805°  \\
2 & \num{7.363e+06} & 48.354°  \\
3 & \num{6.169e+06} & -126.556°  \\
4 & \num{5.445e+06} & -13.976°  \\
5 & \num{6.815e+06} & -111.434°  \\
6 & \num{5.174e+06} & -81.855°  \\
7 & \num{3.962e+06} & 82.760°  \\
8 & \num{3.517e+06} & 119.322°  \\
9 & \num{2.831e+06} & 1.485°  \\
10 & \num{2.291e+06} & 7.703°  \\
11 & \num{2.158e+06} & -53.846°  \\

0 & \num{1.000} & 0.000°  \\
1 & \num{1.463} & -7.095°  \\
2 & \num{1.676} & -26.095°  \\
3 & \num{1.426} & 0.712°  \\
4 & \num{3.797} & -1.052°  \\
5 & \num{3.635} & 26.672°  \\
6 & \num{3.999} & 42.318°  \\
7 & \num{3.703} & -2.503°  \\
8 & \num{3.391} & 49.492°  \\
9 & \num{2.867} & -2.698°  \\
10 & \num{2.751} & 69.663°  \\
11 & \num{2.631} & -40.625°  \\
12 & \num{1.393} & 8.481°  \\
13 & \num{0.987} & -11.289°  \\
14 & \num{1.129} & -1.515°  \\
15 & \num{0.820} & -5.109°  \\
